In [1]:
import sys
sys.path.append('/users/gxb18167/EEG-NLP')

from src import data
from src import utils
from src import Networks

import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

C:\Users\gxb18167\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\gxb18167\Anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
C:\Users\gxb18167\Anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gxb18167\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
train_path = r"C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Named-Entity-Classification\Data-Management\train_NER.pkl"

test_path = r"C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Named-Entity-Classification\Data-Management\test_NER.pkl"

In [3]:
d = data.Data()
util = utils.Utils()

In [4]:
train_NE, train_EEG_segments, train_Classes = d.NER_read_custom_files(train_path)

In [5]:
train_NE[0]

['Henry', 'Ford']

In [6]:
vector_size = 50
#parameters['vector_size'] = vector_size
window = 5
#parameters['window'] = window
min_count = 1
#parameters['min_count'] = min_count
workers = 4

In [7]:
train_word_embeddings, train_NE_embedded = util.NER_Word2Vec(train_NE, vector_size, window, min_count, workers)

In [8]:
len(train_NE_embedded)

585

In [9]:
train_NE_expanded = util.NER_expanded_NER_list(train_EEG_segments, train_NE_embedded)

In [10]:
train_NE_expanded[0][0].shape

(50,)

In [11]:
max_seq_len = max([len(i) for i in train_NE_expanded])
#padding function
for i in range(len(train_NE_expanded)):
    padding_count = max_seq_len - len(train_NE_expanded[i])
    for j in range(padding_count):
        train_NE_expanded[i].append(np.zeros(vector_size))





In [21]:
import torch
from transformers import BertTokenizer, BertModel

class NER_BERT:
    def __init__(self, model_name='bert-base-uncased'):
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = BertModel.from_pretrained(model_name)

    def NER_BERT(self, Word_Labels_List):
        embedded_input = []
        for named_entity in Word_Labels_List:
            inputs = self.tokenizer(named_entity, return_tensors="pt", padding=True, truncation=True)
            outputs = self.model(**inputs)
            last_hidden_states = outputs.last_hidden_state
            embedded_input.append(last_hidden_states.mean(dim=1).squeeze().detach().numpy())

        return embedded_input

In [22]:
ner_bert = NER_BERT()

C:\Users\gxb18167\Anaconda3\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [23]:
embeddings = ner_bert.NER_BERT(train_NE)

In [15]:
len(embeddings[0])

2

In [16]:
train_NE_expanded = util.NER_expanded_NER_list(train_EEG_segments, embeddings, 768)

In [17]:
train_NE_expanded = np.array(train_NE_expanded)

In [18]:
train_NE_expanded.shape




(21875, 7, 768)

In [19]:
print("Type of train_NE_expanded: ", type(train_NE_expanded))
print("Type of train_NE_expanded[0]: ", type(train_NE_expanded[0]))
print("Type of train_NE_expanded[0][0]: ", type(train_NE_expanded[0][0]))

Type of train_NE_expanded:  <class 'numpy.ndarray'>
Type of train_NE_expanded[0]:  <class 'numpy.ndarray'>
Type of train_NE_expanded[0][0]:  <class 'numpy.ndarray'>


In [20]:
train_NE_expanded_tensor = torch.tensor(train_NE_expanded)

In [19]:
expanded_named_entity_list = []
for i in range(len(train_EEG_segments)):
    named_entities = embeddings[i]
    for j in range(len(train_EEG_segments[i])):
        expanded_named_entity_list.append(named_entities)

In [20]:
type(expanded_named_entity_list)

list

In [21]:
desired_shape = (7, 768)
current_shape = expanded_named_entity_list[90].shape

In [22]:
len(expanded_named_entity_list[90].reshape(1,-1).tolist())

1

In [23]:
len(expanded_named_entity_list[0])

2

In [24]:
max_seq_len = max([len(i) for i in expanded_named_entity_list[0]])

In [25]:
max_seq_len

768

In [26]:

max_seq_len = 0



for i in range(len(expanded_named_entity_list)):
    max_seq_len = max(max_seq_len, max_dimension_size(expanded_named_entity_list[i]))


NameError: name 'max_dimension_size' is not defined

In [43]:
max_seq_len

7